## Report2

**Date:** 2024-05-18  
**Author:** Piotr Szepietowski

## External Python Libraries Used

    - Numpy
    - matplotlib

## Jacobi Iterative Method

The Jacobi method is an iterative method for solving systems of linear equations. It consists of the following steps:

1. **Initial condition:** The system of equations must be written in matrix form Ax = b, where:
    - A is the coefficient matrix
    - x is the vector of unknowns
    - b is the vector of constants

2. **System transformation:** For each equation, express the unknown xi through the remaining variables:
    ```
    x₁ = (b₁ - a₁₂x₂ - a₁₃x₃ - ... - a₁ₙxₙ) / a₁₁
    x₂ = (b₂ - a₂₁x₁ - a₂₃x₃ - ... - a₂ₙxₙ) / a₂₂
    ...
    xₙ = (bₙ - aₙ₁x₁ - aₙ₂x₂ - ... - aₙ,ₙ₋₁xₙ₋₁) / aₙₙ
    ```

3. **Iteration:** In each iteration k+1, new values of x are calculated based on values from the previous iteration k:
    ```
    x⁽ᵏ⁺¹⁾ᵢ = (bᵢ - Σⱼ₌₁,ⱼ≠ᵢ aᵢⱼx⁽ᵏ⁾ⱼ) / aᵢᵢ
    ```

4. **Convergence:** The method is convergent when:
    - Matrix A is diagonally dominant
    - Elements on the main diagonal are non-zero

5. **Stopping criterion:** Iterations are performed until a specified accuracy ε is reached:
    ```
    ||x⁽ᵏ⁺¹⁾ - x⁽ᵏ⁾|| < ε
    ```

## Gauss-Seidel Iterative Method

The Gauss-Seidel method is an improved version of the Jacobi method, in which updated values from the current iteration are immediately used to calculate new x values.

1. **Initial condition:** System of equations in the form Ax = b.

2. **System transformation:** Similar to the Jacobi method, each unknown is expressed through the remaining variables.

3. **Iteration:** New x values are calculated sequentially, immediately using the most recent available values:
    ```
    x⁽ᵏ⁺¹⁾ᵢ = (bᵢ - Σⱼ₌₁ⁱ₋₁ aᵢⱼx⁽ᵏ⁺¹⁾ⱼ - Σⱼ₌ᵢ₊₁ⁿ aᵢⱼx⁽ᵏ⁾ⱼ) / aᵢᵢ
    ```

4. **Convergence:** The Gauss-Seidel method is convergent when matrix A is diagonally dominant or positive definite.  
    - **Diagonally dominant matrix:** For each row i:
      $$
      |a_{ii}| > \sum_{j \neq i} |a_{ij}|
      $$
    - **Positive definite matrix:** For every non-zero vector $x$: $x^T A x > 0$.  
    Satisfying one of these conditions guarantees convergence of the method.

5. **Stopping criterion:** Iterations are performed until a specified accuracy ε is reached:
    ```
    ||x⁽ᵏ⁺¹⁾ - x⁽ᵏ⁾|| < ε
    ```

## SOR (Successive Over-Relaxation) Iterative Method

The SOR method is an extension of the Gauss-Seidel method, which introduces a relaxation parameter ω (omega) to accelerate convergence.

1. **Initial condition:** System of equations in the form Ax = b.

2. **System transformation:** Each unknown is expressed through the remaining variables, as in previous methods.

3. **Iteration:** New x values are calculated using the relaxation parameter ω:
    ```
    x⁽ᵏ⁺¹⁾ᵢ = (1 - ω)x⁽ᵏ⁾ᵢ + (ω / aᵢᵢ) * (bᵢ - Σⱼ₌₁ⁱ₋₁ aᵢⱼx⁽ᵏ⁺¹⁾ⱼ - Σⱼ₌ᵢ₊₁ⁿ aᵢⱼx⁽ᵏ⁾ⱼ)
    ```
    where 0 < ω < 2.

4. **Convergence:**  
    - The SOR method is convergent when matrix A is **positive definite** or **strictly diagonally dominant**.  
    - In practice, for diagonally dominant or positive definite matrices, there exists a value of the relaxation parameter ω (usually $1 < \omega < 2$), for which the SOR method converges faster than Gauss-Seidel.  
    - The choice of optimal ω depends on the properties of the matrix - too small or too large values can slow down convergence or even make it impossible.  
    - **Formal conditions:**  
      - If A is a symmetric and positive definite matrix, the SOR method is convergent for $0 < \omega < 2$.  
      - If A is strictly diagonally dominant, the SOR method is also convergent for $0 < \omega < 2$.  
    - For matrices that do not satisfy these conditions, the SOR method may not be convergent regardless of the choice of ω.

5. **Stopping criterion:** Iterations are performed until a specified accuracy ε is reached:
    ```
    ||x⁽ᵏ⁺¹⁾ - x⁽ᵏ⁾|| < ε
    ```

In [ ]:
import numpy as np
import time
from functools import wraps
from concurrent.futures import ThreadPoolExecutor, TimeoutError

timings = {'jacobi': [], 'gauss_seidel': [], 'sor': []}
iterations = {'jacobi': [], 'gauss_seidel': [], 'sor': []}
sizes = []

def record_stats(method_name, size, elapsed, iters):
    timings[method_name].append(elapsed)
    iterations[method_name].append(iters)
    if method_name == 'jacobi':
        sizes.append(size)

def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result, iter_count, printExtended = func(*args, **kwargs)
        end_time = time.time()
        elapsed = end_time - start_time
        method_name = func.__name__
        A = args[0]
        size = A.shape[0]
        record_stats(method_name, size, elapsed, iter_count)
        if printExtended:
            print(f"{method_name} took {elapsed:.4f} seconds to execute")
            print(f"{method_name} solution: {result}")
            print(f"Number of iterations: {iter_count}\n")
        return result
    return wrapper

@timer
def jacobi(A, b, x0=None, tol=1e-10, max_iter=1000, printExtended=True):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()
    D = np.diag(A)
    if np.any(np.abs(D) < 1e-10):
        raise ValueError("Matrix contains zeros on the diagonal")
        
    R = A - np.diagflat(D)
    iter_count = 0
    
    while iter_count < max_iter:
        x_new = (b - np.dot(R, x)) / D
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, iter_count, printExtended
        x = x_new
        iter_count += 1
        
    raise ValueError(f"Failed to converge after {max_iter} iterations")

@timer
def gauss_seidel(A, b, x0=None, tol=1e-10, max_iter=1000, printExtended=True):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()
    iter_count = 0
    if np.any(np.abs(np.diag(A)) < 1e-10):
        raise ValueError("Matrix contains zeros on the diagonal")
    
    while iter_count < max_iter:
        x_new = x.copy()
        for i in range(n):
            sum1 = np.dot(A[i, :i], x_new[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - sum1 - sum2) / A[i, i]
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, iter_count, printExtended
        x = x_new
        iter_count += 1
        
    raise ValueError(f"Failed to converge after {max_iter} iterations")

@timer
def sor(A, b, x0=None, omega=1.0, tol=1e-10, max_iter=1000, printExtended=True):
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()
    iter_count = 0
    if np.any(np.abs(np.diag(A)) < 1e-10):
        raise ValueError("Matrix contains zeros on the diagonal")
    
    while iter_count < max_iter:
        x_new = x.copy()
        for i in range(n):
            sum1 = np.dot(A[i, :i], x_new[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (1 - omega) * x[i] + (omega * (b[i] - sum1 - sum2)) / A[i, i]
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, iter_count, printExtended
        x = x_new
        iter_count += 1
        
    raise ValueError(f"Failed to converge after {max_iter} iterations")

---
## Test on the example from the presentation
Comparison of results, execution times, and the number of required iterations for all three methods using the example given in the presentation:

**Matrix A:**
$$
A = \begin{bmatrix}
3 & -1 & 0 & 0 & 0 & -1 \\
-1 & 3 & -1 & 0 & -1 & 0 \\
0 & -1 & 3 & -1 & 0 & 0 \\
0 & 0 & -1 & 3 & -1 & 0 \\
0 & -1 & 0 & -1 & 3 & -1 \\
-1 & 0 & 0 & 0 & -1 & 3 \\
\end{bmatrix}
$$

**Vector b:**
$$
b = \begin{bmatrix}
0 \\
0 \\
0 \\
0 \\
0 \\
20 \\
\end{bmatrix}
$$

Below are the results, execution times, and number of iterations for Jacobi, Gauss-Seidel, and SOR methods.

In [ ]:
A = np.array([[3,-1,0,0,0,-1], 
                  [-1,3,-1,0,-1,0],
                  [0,-1,3,-1,0,0],
                  [0,0,-1,3,-1,0],
                  [0,-1,0,-1,3,-1],
                  [-1,0,0,0,-1,3]], dtype=float)
b = np.array([0, 0, 0, 0, 0, 20], dtype=float)
print("Solving Ax = b using Jacobi, Gauss-Seidel, and SOR methods:\n")
print(f"Matrix A: {A}")
print(f"Vector b: {b}\n")
jacobi(A, b)
gauss_seidel(A, b)
sor(A, b, omega=1.25)

## Results Comparison 
  
For the example 6x6 matrix, all three methods (Jacobi, Gauss-Seidel, SOR) give very similar results, and the number of iterations and execution time are small:

- **Jacobi:** 111 iterations, time ≈ 0.0033 s  
- **Gauss-Seidel:** 59 iterations, time ≈ 0.0039 s  
- **SOR (ω=1.25):** 31 iterations, time ≈ 0.0011 s  

It is evident that the SOR method is the fastest and requires the fewest iterations, which confirms its advantage when the relaxation parameter is appropriately chosen.  
The Gauss-Seidel method is significantly faster than Jacobi, but slower than SOR.  
For small matrices, the differences are small, but with larger sizes, the advantage of SOR and Gauss-Seidel becomes more apparent.
## Test for larger matrix
We will compare results for an example matrix of size 100x100.

In [ ]:
np.random.seed(42)
n_big = 100
A_big = np.random.rand(n_big, n_big)
for i in range(n_big):
    A_big[i, i] = np.sum(np.abs(A_big[i])) + 1
b_big = np.random.rand(n_big)
print("Solving large Ax = b using Jacobi, Gauss-Seidel, and SOR methods:\n")
print(f"Matrix A_big shape: {A_big.shape}")
print(f"Vector b_big shape: {b_big.shape}\n")
jacobi(A_big, b_big)
gauss_seidel(A_big, b_big)
sor(A_big, b_big, omega=1.1)

## Results Comparison
  
For the 100x100 matrix, the differences between methods become more noticeable:

- **Jacobi:** 616 iterations, time ≈ 0.0112 s  
- **Gauss-Seidel:** 14 iterations, time ≈ 0.0079 s  
- **SOR (ω=1.1):** 17 iterations, time ≈ 0.0139 s  

For larger matrices, the Gauss-Seidel and SOR methods show much better convergence than the Jacobi method. The number of Jacobi iterations grows very quickly, while Gauss-Seidel and SOR achieve a solution in just a few steps. The execution time for all methods remains low, but the advantage of the Gauss-Seidel and SOR methods is clear in terms of iteration count, although a single iteration of these solutions takes noticeably longer than with the Jacobi method.  
In the next steps, a comparison for even larger matrices and a larger number of them will be conducted, and based on this, a graph of the number of iterations versus matrix size and solution time versus matrix size will be created.

In [ ]:
for n_big in range(200, 2001, 100):
    np.random.seed(42)
    A_big = np.random.rand(n_big, n_big)
    for i in range(n_big):
        A_big[i, i] = np.sum(np.abs(A_big[i])) + 1
    b_big = np.random.rand(n_big)
    jacobi(A_big, b_big, printExtended=False, max_iter=float("inf"))
    gauss_seidel(A_big, b_big, printExtended=False)
    sor(A_big, b_big, omega=1.1, printExtended=False)

for n_big in range(2200, 4001, 200):
    np.random.seed(42)
    A_big = np.random.rand(n_big, n_big)
    for i in range(n_big):
        A_big[i, i] = np.sum(np.abs(A_big[i])) + 1
    b_big = np.random.rand(n_big)
    jacobi(A_big, b_big, printExtended=False, max_iter=float("inf"))
    gauss_seidel(A_big, b_big, printExtended=False)
    sor(A_big, b_big, omega=1.1, printExtended=False)

In [ ]:
import matplotlib.pyplot as plt

# Plot timings
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
for method in timings:
    plt.plot(sizes, timings[method], marker='o', label=method.capitalize())
plt.xlabel('Matrix size (n)')
plt.ylabel('Time (seconds)')
plt.title('Execution Time vs Matrix Size')
plt.legend()
plt.grid(True)

# Plot iterations
plt.subplot(1, 2, 2)
for method in iterations:
    plt.plot(sizes, iterations[method], marker='o', label=method.capitalize())
plt.xlabel('Matrix size (n)')
plt.ylabel('Number of Iterations')
plt.title('Iterations vs Matrix Size')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Graph Analysis
With larger matrices, the difference in both execution time and the number of iterations needed becomes very clear.

## Examples of matrices for which these methods don't converge
Examples of matrices for which iterative methods don't converge:

**1. Matrices with zeros on the diagonal**  
Any matrix with zeros on the diagonal immediately causes problems:  
Example:  
$$
A = \begin{bmatrix}
0 & 1 & 2 \\
1 & 2 & 0 \\
3 & 0 & 0 \\
\end{bmatrix}
$$

**2. Matrices that are not positive definite**  
When a symmetric matrix is not positive definite:  
Example:  
$$
A = \begin{bmatrix}
1 & 2 \\
2 & -1 \\
\end{bmatrix}
$$

In [ ]:
class TimeoutException(Exception):
    pass

# Set time limit in seconds
TIMEOUT = 5

def run_with_timeout(func, *args, timeout=TIMEOUT, **kwargs):
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(func, *args, **kwargs)
        try:
            return future.result(timeout=timeout)
        except TimeoutError:
            raise TimeoutException("Time limit exceeded")
        except Exception as e:
            raise e

print("Running tests on problematic matrices...\n")

# 1. Matrix with zeros on the diagonal
print("Test 1: Matrix with zeros on the diagonal")
A_zero_diag = np.array([
    [0, 1, 2],
    [1, 2, 0],
    [3, 0, 0]
], dtype=float)
b_zero_diag = np.array([1, 2, 3], dtype=float)

for method_func, name, kwargs in [
    (jacobi, "Jacobi", {"max_iter": 100}),
    (gauss_seidel, "Gauss-Seidel", {"max_iter": 100}),
    (sor, "SOR", {"omega": 1.1, "max_iter": 100})
]:
    try:
        result = run_with_timeout(method_func, A_zero_diag, b_zero_diag, printExtended=False, **kwargs)
        print(f"{name}: Unexpectedly convergent (should fail with zeros on the diagonal)")
    except TimeoutException as e:
        print(f"{name}: {e}")
    except Exception as e:
        print(f"{name}: {str(e)}")
print()

# 2. Non-positive definite matrix
print("Test 2: Non-positive definite matrix")
A_not_posdef = np.array([
    [1, 2],
    [2, -1]
], dtype=float)
b_not_posdef = np.array([1, 1], dtype=float)

for method_func, name, kwargs in [
    (jacobi, "Jacobi", {"max_iter": 500}),
    (gauss_seidel, "Gauss-Seidel", {"max_iter": 500}),
    (sor, "SOR", {"omega": 1.1, "max_iter": 500})
]:
    try:
        result = run_with_timeout(method_func, A_not_posdef, b_not_posdef, printExtended=False, **kwargs)
        print(f"{name}: Convergent to {result}")
    except TimeoutException as e:
        print(f"{name}: {e}")
    except Exception as e:
        print(f"{name}: {str(e)}")
print()

# 3. Non-diagonally dominant matrix
print("Test 3: Non-diagonally dominant matrix")
A_not_diag_dom = np.array([
    [1, 2],
    [3, 1]
], dtype=float)
b_not_diag_dom = np.array([5, 6], dtype=float)

for method_func, name, kwargs in [
    (jacobi, "Jacobi", {"max_iter": 500}),
    (gauss_seidel, "Gauss-Seidel", {"max_iter": 500}),
    (sor, "SOR", {"omega": 0.5, "max_iter": 500})
]:
    try:
        result = run_with_timeout(method_func, A_not_diag_dom, b_not_diag_dom, printExtended=False, **kwargs)
        print(f"{name}: Convergent to {result}")
    except TimeoutException as e:
        print(f"{name}: {e}")
    except Exception as e:
        print(f"{name}: {str(e)}")
print()

# 4. Ill-conditioned matrix
print("Test 4: Ill-conditioned matrix")
A_ill_cond = np.array([
    [1000, 999],
    [999, 998]
], dtype=float)
b_ill_cond = np.array([1999, 1997], dtype=float)

for method_func, name, kwargs in [
    (jacobi, "Jacobi", {"max_iter": 5000}),
    (gauss_seidel, "Gauss-Seidel", {"max_iter": 5000}),
    (sor, "SOR", {"omega": 0.8, "max_iter": 5000})
]:
    try:
        result = run_with_timeout(method_func, A_ill_cond, b_ill_cond, printExtended=False, **kwargs)
        print(f"{name}: Convergent to {result}")
    except TimeoutException as e:
        print(f"{name}: {e}")
    except Exception as e:
        print(f"{name}: {str(e)}")
print()

# 5. Matrix with complex eigenvalues
print("Test 5: Matrix with complex eigenvalues outside the unit circle")
A_complex = np.array([
    [3, 4],
    [-4, 3]
], dtype=float)
b_complex = np.array([7, -1], dtype=float)

for method_func, name, kwargs in [
    (jacobi, "Jacobi", {"max_iter": 1000}),
    (gauss_seidel, "Gauss-Seidel", {"max_iter": 1000}),
    (sor, "SOR", {"omega": 0.5, "max_iter": 1000})
]:
    try:
        result = run_with_timeout(method_func, A_complex, b_complex, printExtended=False, **kwargs)
        print(f"{name}: Convergent to {result}")
    except TimeoutException as e:
        print(f"{name}: {e}")
    except Exception as e:
        print(f"{name}: {str(e)}")
print()

---
## Graph of convergence rate for given error

In [ ]:
# Test of convergence speed depending on specified error (tolerance)
tolerances = [float("1e-" + str(x)) for x in range(30)]
jacobi_iters_tol, gs_iters_tol, sor_iters_tol = [], [], []
jacobi_times_tol, gs_times_tol, sor_times_tol = [], [], []

n_test = 100
A_test = np.random.rand(n_test, n_test)
for i in range(n_test):
    A_test[i, i] = np.sum(np.abs(A_test[i])) + 1
b_test = np.random.rand(n_test)


for tol in tolerances:
    # Jacobi
    start = time.time()
    try:
        _, iters, _ = jacobi.__wrapped__(A_test, b_test, tol=tol, printExtended=False)
        jacobi_iters_tol.append(iters)
        jacobi_times_tol.append(time.time() - start)
    except Exception:
        jacobi_iters_tol.append(np.nan)
        jacobi_times_tol.append(np.nan)
    # Gauss-Seidel
    start = time.time()
    try:
        _, iters, _ = gauss_seidel.__wrapped__(A_test, b_test, tol=tol, printExtended=False)
        gs_iters_tol.append(iters)
        gs_times_tol.append(time.time() - start)
    except Exception:
        gs_iters_tol.append(np.nan)
        gs_times_tol.append(np.nan)
    # SOR (omega=1.1)
    start = time.time()
    try:
        _, iters, _ = sor.__wrapped__(A_test, b_test, omega=1.1, tol=tol, printExtended=False)
        sor_iters_tol.append(iters)
        sor_times_tol.append(time.time() - start)
    except Exception:
        sor_iters_tol.append(np.nan)
        sor_times_tol.append(np.nan)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(tolerances, jacobi_iters_tol, marker='o', label='Jacobi')
plt.plot(tolerances, gs_iters_tol, marker='o', label='Gauss-Seidel')
plt.plot(tolerances, sor_iters_tol, marker='o', label='SOR (ω=1.1)')
plt.xscale('log')
plt.xlabel('Tolerance (error)')
plt.ylabel('Number of iterations')
plt.title('Number of iterations vs tolerance')
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(tolerances, jacobi_times_tol, marker='o', label='Jacobi')
plt.plot(tolerances, gs_times_tol, marker='o', label='Gauss-Seidel')
plt.plot(tolerances, sor_times_tol, marker='o', label='SOR (ω=1.1)')
plt.xscale('log')
plt.xlabel('Tolerance (error)')
plt.ylabel('Execution time (s)')
plt.title('Execution time vs tolerance')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

---
## Graph of the effect of relaxation parameter on SOR method convergence

In [ ]:
omegas = np.arange(0.1, 2.0, 0.05)
sor_iters = []
sor_times = []

# We'll use A_big and b_big from previous tests
n_test = 100
np.random.seed(42)
A_test = np.random.rand(n_test, n_test)
for i in range(n_test):
    A_test[i, i] = np.sum(np.abs(A_test[i])) + 1
b_test = np.random.rand(n_test)

for omega in omegas:
    try:
        start = time.time()
        _, iters, _ = sor.__wrapped__(A_test, b_test, omega=omega, printExtended=False)
        elapsed = time.time() - start
        sor_iters.append(iters)
        sor_times.append(elapsed)
    except Exception:
        sor_iters.append(np.nan)
        sor_times.append(np.nan)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(omegas, sor_iters, marker='o')
plt.xlabel('Relaxation parameter ω')
plt.ylabel('Number of iterations to convergence')
plt.title('Effect of ω on number of SOR iterations')
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(omegas, sor_times, marker='o', color='orange')
plt.xlabel('Relaxation parameter ω')
plt.ylabel('Execution time (s)')
plt.title('Effect of ω on SOR execution time')
plt.grid(True)

plt.tight_layout()
plt.show()